In [1]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc

In [2]:
df = pd.read_csv("E:/accident_severity_analytics/data/processed/Cleaned_US_Accidents.csv", parse_dates=["Start_Time", "End_Time", "Weather_Timestamp"])
df

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Duration
0,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.010,Accident,state route 32,...,False,False,False,True,False,Night,Night,Day,Day,30.000000
1,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.010,Accident,i-75,...,False,False,False,False,False,Night,Day,Day,Day,30.000000
2,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.010,Accident,miamisburg centerville road,...,False,False,False,True,False,Day,Day,Day,Day,30.000000
3,A-6,Source2,3,2016-02-08 07:44:26,2016-02-08 08:14:26,40.100590,-82.925194,0.010,Accident,westerville road,...,False,False,False,False,False,Day,Day,Day,Day,30.000000
4,A-7,Source2,2,2016-02-08 07:59:35,2016-02-08 08:29:35,39.758274,-84.230507,0.000,Accident,woodward avenue,...,False,False,False,False,False,Day,Day,Day,Day,30.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6335823,A-7777757,Source1,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379360,0.543,Accident,pomona fwy,...,False,False,False,False,False,Day,Day,Day,Day,28.600000
6335824,A-7777758,Source1,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148060,0.338,Accident,i-8,...,False,False,False,False,False,Day,Day,Day,Day,26.883333
6335825,A-7777759,Source1,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847790,0.561,Accident,garden grove fwy,...,False,False,False,False,False,Day,Day,Day,Day,28.466667
6335826,A-7777760,Source1,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.992460,-118.403020,0.772,Accident,san diego fwy,...,False,False,False,False,False,Day,Day,Day,Day,29.350000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335828 entries, 0 to 6335827
Data columns (total 42 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     object        
 1   Source                 object        
 2   Severity               int64         
 3   Start_Time             datetime64[ns]
 4   End_Time               datetime64[ns]
 5   Start_Lat              float64       
 6   Start_Lng              float64       
 7   Distance(mi)           float64       
 8   Description            object        
 9   Street                 object        
 10  City                   object        
 11  County                 object        
 12  State                  object        
 13  Zipcode                object        
 14  Country                object        
 15  Timezone               object        
 16  Weather_Timestamp      datetime64[ns]
 17  Temperature(F)         float64       
 18  Humidity(%)           

In [4]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])
app.title = "US Accidents Dashboard"

# ======================= Layout =======================
app.layout = dbc.Container([
    html.H1("US Accidents Dashboard", className="text-center mt-4 mb-4"),

    dbc.Row([
        dbc.Col([
            html.Label("Select State"),
            dcc.Dropdown(
                options=[{'label': s, 'value': s} for s in sorted(df['State'].unique())],
                value='CA',
                id='state-dropdown',
                multi=False
            )
        ], md=3),
        dbc.Col([
            html.Label("Select Severity"),
            dcc.Dropdown(
                options=[{'label': str(s), 'value': s} for s in sorted(df['Severity'].unique())],
                value=2,
                id='severity-dropdown',
                multi=False
            )
        ], md=3),
        dbc.Col([
            html.Label("Weather Condition"),
            dcc.Dropdown(
                options=[{'label': str(w), 'value': w} for w in df['Weather_Condition'].dropna().unique()],
                id='weather-dropdown',
                multi=False,
                placeholder="Select weather condition"
            )
        ], md=3),
        dbc.Col([
            html.Label("Select Date Range"),
            dcc.DatePickerRange(
                id='date-picker',
                start_date=df['Start_Time'].min().date(),
                end_date=df['Start_Time'].max().date()
            )
        ], md=3),
    ], className="mb-3"),

    html.Div([
        dbc.Button("Refresh Filters", id="refresh-btn", color="primary", className="mb-3")
    ], className="text-end"),

    # Map graph showing accident locations
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='map-graph')
        ], md=12),
    ]),

    # Bar chart for top 10 cities & Pie chart for weather distribution
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='bar-graph')
        ], md=6),
        dbc.Col([
            dcc.Graph(id='pie-graph')
        ], md=6),
    ]),

    # Line chart for accidents per hour
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='line-graph')
        ], md=12),
    ])
], fluid=True)

# ======================= Callbacks =======================
@app.callback(
    [Output('map-graph', 'figure'),
     Output('bar-graph', 'figure'),
     Output('pie-graph', 'figure'),
     Output('line-graph', 'figure'),
     Output('state-dropdown', 'value'),
     Output('severity-dropdown', 'value'),
     Output('weather-dropdown', 'value'),
     Output('date-picker', 'start_date'),
     Output('date-picker', 'end_date')],
    [Input('state-dropdown', 'value'),
     Input('severity-dropdown', 'value'),
     Input('weather-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date'),
     Input('refresh-btn', 'n_clicks')]
)
def update_graphs(state, severity, weather, start_date, end_date, n_clicks):
    ctx = dash.callback_context

    if ctx.triggered and ctx.triggered[0]['prop_id'] == 'refresh-btn.n_clicks':
        state = 'CA'
        severity = 2
        weather = None
        start_date = df['Start_Time'].min().date()
        end_date = df['Start_Time'].max().date()

    # Filter dataframe based on inputs
    dff = df.copy()
    dff = dff[dff['State'] == state]
    dff = dff[dff['Severity'] == severity]
    if weather:
        dff = dff[dff['Weather_Condition'] == weather]
    dff = dff[(dff['Start_Time'] >= start_date) & (dff['Start_Time'] <= end_date)]

    # Scatter map using scatter_map instead of deprecated scatter_mapbox
    map_fig = px.scatter_map(
        dff.head(1000),  # limit to first 1000 records for performance
        lat="Start_Lat", lon="Start_Lng",
        color="Severity",
        hover_name="City",
        hover_data=["Start_Time", "Weather_Condition"],
        zoom=4,
        height=500
    )
    map_fig.update_layout(map_style="open-street-map", margin={"r":0,"t":0,"l":0,"b":0})

    # Bar chart: Top 10 cities with most accidents
    bar_fig = px.bar(
        dff['City'].value_counts().nlargest(10),
        labels={'index': 'City', 'value': 'Accident Count'},
        title='Top 10 Cities with Most Accidents'
    )

    # Pie chart: Distribution of weather conditions
    pie_fig = px.pie(
        dff,
        names='Weather_Condition',
        title='Weather Conditions Distribution'
    )

    # Add hour column for line chart
    dff['Hour'] = dff['Start_Time'].dt.hour
    line_fig = px.line(
        dff.groupby('Hour').size().reset_index(name='Count'),
        x='Hour', y='Count',
        title='Accidents per Hour'
    )

    return map_fig, bar_fig, pie_fig, line_fig, state, severity, weather, start_date, end_date

# ======================= Run App =======================
app.run(jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/


In [5]:
print(df['Visibility(mi)'].head(1))

0    10.0
Name: Visibility(mi), dtype: float64
